### 단어 수준 임베딩 예시

In [85]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
import os
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

os.chdir("/content/drive/My Drive")
data = pd.read_csv("jobkorea_all.csv")

In [91]:
from preprocess import _preprocess_company, _preprocess_qna

# 회사명, 답변 전처리
data = _preprocess_company(data)
data = _preprocess_qna(data)

In [92]:
# 회사명을 단어 사전에 추가
company_dict = pd.DataFrame({"회사명": data["회사명"].unique()})
company_dict["품사"] = "N"
company_dict.to_csv("company_dict.txt", sep="\t", index=False, header=False)

In [ ]:
!pip install konlpy

In [95]:
from konlpy.tag import Komoran

komoran = Komoran(userdic="company_dict.txt")

In [96]:
# 형태소 분리
data["답변"] = data["답변"].apply(lambda x: komoran.morphs(x))
data["답변"].head()

0    [능동, 적, 이, ㄴ, 직무, 수행, 의, 기회, 코리아세븐, 유통업, 에, 대하...
1    [내, 가족, 내, 물건, 같이, 생각, 하, 는, 책임감, 대학교, 에서, 경제학...
2    [열정, 과, 창의, 능동, 성, 을, 통하, ㄴ, 성과, 직접, 제안, 하, ㄴ,...
3    [역량, 발전, 을, 위하, ㄴ, 노력, 데이터, 분석, 능력, 소통, 능력, 코리...
4    [편안, 하, ㄴ, 서비스, 상권, 과, 시즌, 에, 맞, 는, 운영, 최근, 세븐...
Name: 답변, dtype: object

FastText, Word2Vec

In [97]:
from gensim.models import Word2Vec, FastText

size =  100 # 단어 벡터 차원 수
window = 5 # 함께 고려할 주변 단어 개수
sg = 0 # 0이면 skipgram 방식, 1이면 cbow 방식
min_count = 1 # 일정 개수 이상 등장하는 단어에 대해서만 벡터로 만듦
alpha = 0.05 # 단어 벡터로 만들기 위한 학습 과정에 쓰이는 학습률 (learning rate)
min_alpha = 1e-4 # 학습시키며 학습률을 점차 줄여가는데, 이때 학습률의 최소값
batch_words = 512 # 학습 과정에서 한 번에 학습시킬 단어 개수

In [ ]:
ft_cb = FastText(data["답변"].to_list(), size=size, sg=sg, alpha=alpha, window=window, min_count=min_count, min_alpha=min_alpha, batch_words=batch_words)
ft_sg = FastText(data["답변"].to_list(), size=size, sg=sg, alpha=alpha, window=window, min_count=min_count, min_alpha=min_alpha, batch_words=batch_words)
wv_cb = Word2Vec(data["답변"].to_list(), size=size, sg=sg, alpha=alpha, window=window, min_count=min_count, min_alpha=min_alpha, batch_words=batch_words)
wv_sg = Word2Vec(data["답변"].to_list(), size=size, sg=sg, alpha=alpha, window=window, min_count=min_count, min_alpha=min_alpha, batch_words=batch_words)

In [ ]:
ft_cb.wv.similar_by_word("고객")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('소비자', 0.7930922508239746),
 ('소비자학', 0.7127760648727417),
 ('손님', 0.6703469753265381),
 ('사용자', 0.6609821915626526),
 ('승객', 0.6604449152946472),
 ('고객만족센터', 0.6349461674690247),
 ('고객감동', 0.625013530254364),
 ('고용자', 0.6241613030433655),
 ('조합원', 0.6187136173248291),
 ('상대방', 0.618556022644043)]

In [ ]:
ft_sg.wv.similar_by_word("고객")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('원츠를', 0.6693088412284851),
 ('서비스', 0.6591088175773621),
 ('손님', 0.6547598838806152),
 ('양방', 0.654032826423645),
 ('개인의', 0.6507511138916016),
 ('맞춤화', 0.6497688293457031),
 ('소비자', 0.6395623087882996),
 ('고객감동', 0.6350874304771423),
 ('원츠까지', 0.6341608762741089),
 ('조합원', 0.6314752101898193)]

In [ ]:
wv_cb.wv.similar_by_word("고객")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('소비자', 0.7776912450790405),
 ('손님', 0.6735745668411255),
 ('사용자', 0.645987331867218),
 ('승객', 0.6387404203414917),
 ('민원인', 0.6338218450546265),
 ('화주', 0.6227380633354187),
 ('이해관계자', 0.5914591550827026),
 ('상대방', 0.5861569046974182),
 ('임직원', 0.567523181438446),
 ('시청자', 0.5562482476234436)]

In [ ]:
wv_sg.wv.similar_by_word("고객")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('원츠를', 0.7047795653343201),
 ('손님', 0.6841558218002319),
 ('서비스', 0.667316198348999),
 ('도제', 0.6575407981872559),
 ('응대', 0.6516178846359253),
 ('소비자', 0.6502076387405396),
 ('행원', 0.6493786573410034),
 ('unmet', 0.6473048329353333),
 ('원츠까지', 0.6405093669891357),
 ('하대', 0.6368513107299805)]

In [ ]:
print(len(wv_sg.wv.vocab)) # 단어 벡터로 등록된 단어 개수

38428


GloVe

In [ ]:
!pip install glove_python

In [101]:
from glove import Corpus, Glove

corpus = Corpus()
corpus.fit(data["답변"], window=5)

In [116]:
glove = Glove(no_components=100, learning_rate=0.05) # no_components: 단어 벡터 차원 수
glove.fit(corpus.matrix, epochs=10, no_threads=4, verbose=False)
glove.add_dictionary(corpus.dictionary)

In [117]:
glove.most_similar("고객")

[('평생', 0.838656169447482),
 ('단골', 0.836212777733712),
 ('중심', 0.8004784988793258),
 ('응대', 0.7964313616872021)]

In [118]:
glove.most_similar("인공지능")

[('패션', 0.8629832374592586),
 ('핀테크', 0.8491576240337309),
 ('iot', 0.8459145746952812),
 ('화장품', 0.8394349408899434)]

In [119]:
len(glove.dictionary) # 단어 벡터로 등록된 단어 개수

39463